# Distribution 1
Williams, H. Paul. Model building in mathematical programming. John Wiley & Sons, 2013.

In [1]:
from typing import Dict

import numpy as np
import pandas as pd
from pulp import LpProblem, LpMaximize, LpMinimize, LpVariable, lpSum

## 集合

In [2]:
factories = ["Liverpool", "Brighton"]
depots = ["Newcastle", "Birmingham", "London", "Exeter"]
customers = [f"C{i}" for i in range(1, 7)]
suppliers = factories + depots
recievers = depots + customers

## パラメータ

In [3]:
M = 1e+3
trans_uc = pd.DataFrame([
    [0.5, M, M, M, M, M],
    [0.5, 0.3, M, M, M, M],
    [1, 0.5, M, M, M, M],
    [0.2, 0.2, M, M, M, M],
    [1, 2, M, 1, M, M],
    [M, M, 1.5, 0.5, 1.5, M],
    [1.5, M, 0.5, 0.5, 2, 0.2],
    [2, M, 1.5, 1, M, 1.5],
    [M, M, M, .5, .5, .5],
    [1, M, 1, M, 1.5, 1.5]
], index=depots + customers, columns=suppliers)
trans_uc

,Liverpool,Brighton,Newcastle,Birmingham,London,Exeter
Newcastle,0.5,1000.0,1000.0,1000.0,1000.0,1000.0
Birmingham,0.5,0.3,1000.0,1000.0,1000.0,1000.0
London,1.0,0.5,1000.0,1000.0,1000.0,1000.0
Exeter,0.2,0.2,1000.0,1000.0,1000.0,1000.0
C1,1.0,2.0,1000.0,1.0,1000.0,1000.0
C2,1000.0,1000.0,1.5,0.5,1.5,1000.0
C3,1.5,1000.0,0.5,0.5,2.0,0.2
C4,2.0,1000.0,1.5,1.0,1000.0,1.5
C5,1000.0,1000.0,1000.0,0.5,0.5,0.5
C6,1.0,1000.0,1.0,1000.0,1.5,1.5


In [4]:
preference = pd.DataFrame([
    [1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 1],
], index=customers, columns=suppliers)
preference

,Liverpool,Brighton,Newcastle,Birmingham,London,Exeter
C1,1,0,0,0,0,0
C2,0,0,1,0,0,0
C3,0,0,0,0,0,0
C4,0,0,0,0,0,0
C5,0,0,0,1,0,0
C6,0,0,0,0,1,1


In [5]:
capacities = pd.Series([150000, 200000], index=factories)
throughput_ub = pd.Series([70000, 50000, 100000, 40000], index=depots)
requirements = pd.Series([50000, 10000, 40000, 35000, 60000, 20000], index=customers)

## 変数

In [6]:
trans = pd.DataFrame(LpVariable.dicts("transport_amount", (suppliers, recievers),  lowBound=0, upBound=None))
trans

,Liverpool,Brighton,Newcastle,Birmingham,London,Exeter
Newcastle,transport_amount_Liverpool_Newcastle,transport_amount_Brighton_Newcastle,transport_amount_Newcastle_Newcastle,transport_amount_Birmingham_Newcastle,transport_amount_London_Newcastle,transport_amount_Exeter_Newcastle
Birmingham,transport_amount_Liverpool_Birmingham,transport_amount_Brighton_Birmingham,transport_amount_Newcastle_Birmingham,transport_amount_Birmingham_Birmingham,transport_amount_London_Birmingham,transport_amount_Exeter_Birmingham
London,transport_amount_Liverpool_London,transport_amount_Brighton_London,transport_amount_Newcastle_London,transport_amount_Birmingham_London,transport_amount_London_London,transport_amount_Exeter_London
Exeter,transport_amount_Liverpool_Exeter,transport_amount_Brighton_Exeter,transport_amount_Newcastle_Exeter,transport_amount_Birmingham_Exeter,transport_amount_London_Exeter,transport_amount_Exeter_Exeter
C1,transport_amount_Liverpool_C1,transport_amount_Brighton_C1,transport_amount_Newcastle_C1,transport_amount_Birmingham_C1,transport_amount_London_C1,transport_amount_Exeter_C1
C2,transport_amount_Liverpool_C2,transport_amount_Brighton_C2,transport_amount_Newcastle_C2,transport_amount_Birmingham_C2,transport_amount_London_C2,transport_amount_Exeter_C2
C3,transport_amount_Liverpool_C3,transport_amount_Brighton_C3,transport_amount_Newcastle_C3,transport_amount_Birmingham_C3,transport_amount_London_C3,transport_amount_Exeter_C3
C4,transport_amount_Liverpool_C4,transport_amount_Brighton_C4,transport_amount_Newcastle_C4,transport_amount_Birmingham_C4,transport_amount_London_C4,transport_amount_Exeter_C4
C5,transport_amount_Liverpool_C5,transport_amount_Brighton_C5,transport_amount_Newcastle_C5,transport_amount_Birmingham_C5,transport_amount_London_C5,transport_amount_Exeter_C5
C6,transport_amount_Liverpool_C6,transport_amount_Brighton_C6,transport_amount_Newcastle_C6,transport_amount_Birmingham_C6,transport_amount_London_C6,transport_amount_Exeter_C6


In [7]:
trans_cost = (trans * trans_uc).values.sum()
preference_reward = 0.01 * (preference * trans.loc[customers]).values.sum()
total_cost = trans_cost - preference_reward

In [8]:
model = LpProblem("Distribution1", LpMinimize)
model.setObjective(total_cost)

### 制約条件

In [9]:
# requirements
recv = trans.loc[customers].sum(axis=1)
for c in customers:
    model.addConstraint(recv[c] == requirements[c])

In [10]:
# capacity
send = trans.loc[:, factories].sum(axis=0)
for f in factories:
    model.addConstraint(send[f] <= capacities[f])

In [11]:
# throughtput
transin = trans.loc[depots].sum(axis=1)
transout = trans.loc[:, depots].sum(axis=0)
for d in depots:
    model.addConstraint(transout[d] <= throughput_ub[d])    
    model.addConstraint(transin[d] <= throughput_ub[d])    

In [12]:
model.solve()

1

In [13]:
trans_cost.value()

148000.0